In [1]:
import json
import requests
import datetime

root = 'http://88.169.46.187:9200'
headers = {'Content-Type':'Application/json'}

In [17]:
# create the mapping for our data
# for the search engine i don't change the type of date, as I need to search it as a text/keyword
requests.put(root + '/engine-recherche',headers=headers, data=json.dumps(
        {
            "settings": {      
              "analysis": {
                "normalizer": {
                  "lowercase_normalizer": {
                    "type": "custom",
                    "char_filter": [],
                    "filter": ["lowercase"]
                  }
                }     
              }
            },
            "mappings": {
            "_default_": {
                "numeric_detection": 'true',
                "date_detection": 'true' , 
                 "dynamic_templates": [
                  {
                  "lesnames": {
                    "path_match": "*name",
                    "mapping": {
                      "type": "keyword",
                      "normalizer": "lowercase_normalizer"
                    }
                  }
                },
                {
                  "lesdates": {
                    "path_match": "*date",
                    "mapping": {
                      "type": "keyword"
                    }
                  }
                },
                {
                  "lereste": { 
                    "match_mapping_type": "string",
                    "mapping": {
                      "type": "text",
                      "normalizer": "lowercase_normalizer"
                    }
                  }
                }
              ]
            
            }
          }
        }
)).json()

{'acknowledged': True,
 'index': 'engine-recherche',
 'shards_acknowledged': True}

In [18]:
# index of data
headers = {'Content-Type':'Application/json'}
mydata =""
madataset = open('commitsAllTable.json', 'r', 1024*1024*16)
i = 1
for line in madataset: 
    jsonLine = json.loads(line)
    jsonLine["author_date"] = str(jsonLine["author_date"]).replace(" UTC", "")
    jsonLine["committer_date"] = str(jsonLine["committer_date"]).replace(" UTC", "")
    fullLine = json.dumps({"index": {"_index": "engine-recherche", "_type":"data", "_id": i}}) + '\n' + json.dumps(jsonLine) + '\n'
    i = i + 1
    mydata = mydata + fullLine
    if ((i % 10000) == 0):
        requests.post(root + '/_bulk', headers=headers, data=mydata.encode('utf-8'))
        print(str(datetime.datetime.now()) + ", " + str(i))
        mydata = ""
        
requests.post(root + '/_bulk', headers=headers, data=mydata.encode('utf-8'))

2018-03-16 23:04:57.694661, 10000
2018-03-16 23:06:13.444277, 20000
2018-03-16 23:07:35.252973, 30000
2018-03-16 23:08:58.106885, 40000
2018-03-16 23:11:06.909106, 50000
2018-03-16 23:12:38.487244, 60000


<Response [200]>

In [16]:
# if i have problem so i should delete the index that i have created in order to recreate the new one with the same name
requests.delete(root + '/engine-recherche')

<Response [200]>

In [10]:
requests.put(root + '/contents',headers=headers, data=json.dumps(
        {
            "settings": {      
              "analysis": {
                "normalizer": {
                  "lowercase_normalizer": {
                    "type": "custom",
                    "char_filter": [],
                    "filter": ["lowercase"]
                  }
                }     
              }
            },
            "mappings": {
            "_default_": {
                "numeric_detection": 'true',
                "date_detection": 'true' , 
                 "dynamic_templates": [
                  {
                  "lesnames": {
                    "path_match": "*name",
                    "mapping": {
                      "type": "keyword",
                      "normalizer": "lowercase_normalizer"
                    }
                  }
                },
                {
                  "lereste": { 
                    "match_mapping_type": "string",
                    "mapping": {
                      "type": "text",
                      "normalizer": "lowercase_normalizer"
                    }
                  }
                }
              ]
            
            }
          }
        }
)).json()

{'acknowledged': True, 'index': 'contents', 'shards_acknowledged': True}

In [11]:
# for contentsTable 
headers = {'Content-Type':'Application/json'}
mydata =""
madataset = open('contentsTable.json', 'r', 1024*1024*16)
i = 1
for line in madataset: 
    fullLine = json.dumps({"index": {"_index": "contents", "_type":"data", "_id": i}}) + '\n' + line + '\n'
    i = i + 1
    mydata = mydata + fullLine
    if ((i % 10000) == 0):
        requests.post(root + '/_bulk', headers=headers, data=mydata.encode('utf-8'))
        print(str(datetime.datetime.now()) + ", " + str(i))
        mydata = ""
        
requests.post(root + '/_bulk', headers=headers, data=mydata.encode('utf-8'))

2018-03-16 16:39:18.806171, 10000
2018-03-16 16:55:37.887625, 20000


In [9]:
requests.delete(root + '/contents')

<Response [200]>